In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import glob
import os
from multiprocessing import Process
import sys
sys.path.append('..')
from modules import utils
from modules import constants
from sklearn.impute import KNNImputer

In [12]:
seed = constants.SEED
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED']=str(seed)
tf.set_random_seed(seed)
tf.compat.v1.set_random_seed(seed)

42

In [ ]:
constants.BETA, constants.CHECKPOINT_FREQ

In [ ]:
def run_dqn_model(model_type, steps):
    
    dir_name = f'seed_{seed}_{steps}'
    parent_dir = f'../models/logs/{model_type}/missingness/0.1/biopsy_9/knn_imputer/default_mean_k_1'
    path = os.path.join(parent_dir, dir_name)
    os.mkdir(path)
  
    if model_type=='dqn':
        model = utils.stable_vanilla_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dqn', filename=f'dqn_{steps}')
    elif model_type=='ddqn':
        model = utils.stable_double_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='ddqn', filename=f'ddqn_{steps}')
    elif model_type== 'dueling_dqn':
        model = utils.stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn', filename=f'dueling_dqn_{steps}')
    elif model_type == 'dueling_ddqn':
        model = utils.stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn', filename=f'dueling_ddqn_{steps}')
    elif model_type =='dqn_per':
        model = utils.stable_vanilla_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dqn_per', filename=f'dqn_per_{steps}', per=True)
    elif model_type == 'ddqn_per':
        model = utils.stable_double_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='ddqn_per', filename=f'ddqn_per_{steps}', per=True)
    elif model_type == 'dueling_dqn_per':
        model = utils.stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn_per', filename=f'dueling_dqn_per_{steps}', per=True)
    elif model_type == 'dueling_ddqn_per':
        model = utils.stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn_per', filename=f'dueling_ddqn_per_{steps}', per=True)
    else:
        raise ValueError(f'Unknown model type - {model_type}!')
    return model

In [ ]:
train_df = pd.read_csv('../new_data/knn_imputed/default_mean_k_1/missingness_0.1.csv')
# train_df = train_df.fillna(-1)
train_df.head()

In [ ]:
train_df.label.value_counts()

In [ ]:
train_df.iloc[90]

In [ ]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

In [ ]:
# model_names = ['dqn', 'ddqn', 'dueling_dqn', 'dueling_ddqn', 'dqn_per', 'ddqn_per', 'dueling_dqn_per', 
#                'dueling_ddqn_per']
# model_names = ['dueling_dqn_per', 'dueling_ddqn_per']
model_name = 'dueling_dqn_per'
seeds = [42, 63, 84, 105, 126]
procs = []
steps = int(100e6)

In [ ]:
# run_dqn_model(model_names[0],steps)

In [ ]:
# for name in model_names:
for seed in seeds:
#     run_dqn_model(name, steps)
    proc = Process(target=run_dqn_model, args=(model_name, steps, seed))
    procs.append(proc)
    proc.start()

In [ ]:
for proc in procs:
    proc.join()
print('All jobs completed and terminated successfully')

In [ ]:
# mylist = [{'name':30, 'list_feature': np.array([1,2,3]), 'label':0}, {'name':50, 'list_feature': np.array
#                                                                       ([4,5,6]), 'label':1}]
# a = pd.DataFrame(mylist)
# a

In [ ]:
# a.iloc[0]['list_feature']

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# X = a[['name', 'list_feature']]
# y = a['label']
# rf = RandomForestClassifier().fit(X, y)